In [2]:
import pandas as pd
from pathlib import Path
import tempfile
import io
from mrich import print
from datetime import datetime
import subprocess

### baseline - 25 wks

In [4]:
mdf1 = pd.read_excel("data/InVivo2/JM_InVivo2_EchoRandomisation_max.xlsx", usecols="B:G", skiprows=1)
# df = df.set_index(["Timepoint", "Animal number", "Rat"])
indices = ["Timepoint", "Treatment", "Rat", "Animal number"]
mdf1 = mdf1.set_index("Randomised number")
mdf1.loc[mdf1["Rat"] == "NM", "Rat"] = 79
mdf1["Treatment"] = mdf1["Rat"].apply(lambda x: "Control" if x%4<2 else "IR")
mdf1

,Timepoint,Animal number,Rat,Ear Notch,Acquisition Date,Treatment
Randomised number,,,,,,
53,Baseline,M00681267,64,L,2024-08-31,Control
47,Baseline,M00681268,65,R,2024-08-31,Control
128,Baseline,M00681269,66,LL,2024-08-31,IR
52,Baseline,M00681270,67,NM,2024-09-02,IR
40,Baseline,M00681271,68,L,2024-08-31,Control
...,...,...,...,...,...,...
74,25 wk,M00681278,75,NM,2025-03-28,IR
57,25 wk,M00681279,76,L,2025-03-28,Control
28,25 wk,M00681280,77,R,2025-03-28,Control


In [5]:
data = {}

### PSLAX Bmode

keys = [
    "PSLAX_Bmode",
    "PSLAX_Mmode",
    "PSSAX_Mmode",
]

for key in keys:
    for file in Path("data/InVivo2/blinded_echo").glob(f"202?????_InVivo2_Blinded_*_{key}.csv"):
        
        date, _, _, image_num, _, _ = file.name.split("_")
        
        image_num = int(image_num)
        
        match key:
            case "PSLAX_Bmode":
                search = '"Measurement","Mode","Parameter","Units",'
                n_lines = 11
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                search = '"Measurement","Mode","Parameter","Units","Avg","STD","Instance 1","Instance 2","Instance 3"'
                n_lines = 6
            case _:
                raise ValueError

        cmds = [
            "grep",
            f"-A{n_lines}",
            search,
            file,
        ]

        result = subprocess.run(cmds, stdout=subprocess.PIPE, text=True)
        df = pd.read_csv(io.StringIO(result.stdout), index_col=False)

        d = data.setdefault(image_num, {})
        
        d.update({
            "Randomised number":int(image_num),
            f"{key}: Date": datetime.strptime(date, "%Y%m%d").date(),
        })

        match key:
            case "PSLAX_Bmode":
                for i,row in df.iterrows():
                    param = row["Parameter"]
                    unit = row["Units"]
                    value = row[-1]
                    d[f"{key}: {param} [{unit}]"] = value
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                for i,row in df.iterrows():
                    param = row["Measurement"]
                    unit = row["Units"]
                    d[f"{key}: {param} Avg [{unit}]"] = row["Avg"]
                    d[f"{key}: {param} STD [{unit}]"] = row["STD"]

#### COMBINE

df1 = pd.DataFrame(data.values())
# df1 = df.set_index("Randomised number")

#### MAP TO RANDOM NUMBERS

for col in indices:
    df1[col] = df1["Randomised number"].apply(lambda x: mdf1.loc[x,col])

#### SORT

timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk", "30 wk", "36 wk"]

df1["Timepoint"] = pd.Categorical(df1["Timepoint"], categories=timepoints, ordered=True)
    
df1 = df1.sort_values(by=indices)
df1 = df1.set_index(indices + ["Randomised number"])

df1

PSLAX_Bmode: Date  \
Timepoint Treatment Rat Animal number Randomised number                     
Baseline  Control   64  M00681267     53                       2025-04-28   
                    65  M00681268     47                       2025-04-26   
                    68  M00681271     40                       2025-04-26   
                    69  M00681272     117                      2025-05-02   
                    72  M00681275     76                       2025-04-28   
...                                                                   ...   
25 wk     IR        71  M00681274     18                       2025-04-25   
                    74  M00681277     65                       2025-04-28   
                    75  M00681278     74                       2025-04-28   
                    78  M00681281     21                       2025-04-25   
                    79  M00681282     124                      2025-05-02   

                                                         PSLAX_Bmode: Heart Rate [BPM]  \
Timepoint Treatment Rat Animal number Randomised number                                  
Baseline  Control   64  M00681267     53                                    428.189117   
                    65  M00681268     47                                    435.808970   
                    68  M00681271     40                                    468.566966   
                    69  M00681272     117                                   418.628990   
                    72  M00681275     76                                    399.600400   
...                                                                                ...   
25 wk     IR        71  M00681274     18                                    345.622120   
                    74  M00681277     65                                    352.164343   
                    75  M00681278     74                                    370.084811   
                    78  M00681281     21                                    351.699883   
                    79  M00681282     124                                   361.391357   

                                                         PSLAX_Bmode: Area [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                            
Baseline  Control   64  M00681267     53                               30.421540   
                    65  M00681268     47                               32.815279   
                    68  M00681271     40                               28.106887   
                    69  M00681272     117                              34.847759   
                    72  M00681275     76                               29.239489   
...                                                                          ...   
25 wk     IR        71  M00681274     18                               35.601831   
                    74  M00681277     65                               70.887392   
                    75  M00681278     74                               38.012953   
                    78  M00681281     21                               25.901006   
                    79  M00681282     124                              40.650569   

                                                         PSLAX_Bmode: Area;s [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                              
Baseline  Control   64  M00681267     53                                 26.736552   
                    65  M00681268     47                                 30.244550   
                    68  M00681271     40                                 26.581895   
                    69  M00681272     117                                33.943376   
                    72  M00681275     76                                 27.919859   
...                                                                            ...   
25 wk     IR        71  M00681274     18                                 32.689276   
                

### 30-36 weeks

In [6]:
mdf2 = pd.read_excel("data/30-36wk Blinded/JM_InVivo2_30-36wk_EchoRandomisation_PG.xlsx", usecols="A:F", skiprows=1)
# df = df.set_index(["Timepoint", "Animal number", "Rat"])
indices = ["Timepoint", "Treatment", "Rat", "Animal number"]
mdf2 = mdf2.set_index("Randomised number")
mdf2.loc[mdf2["Rat"] == "NM", "Rat"] = 79
mdf2["Treatment"] = mdf2["Rat"].apply(lambda x: "Control" if x%4<2 else "IR")
mdf2

,Timepoint,Animal number,Rat,Ear Notch,Acquisition Date,Treatment
Randomised number,,,,,,
11,30 wk,M00681267,64,L,2025-05-01,Control
10,30 wk,M00681268,65,R,2025-05-01,Control
8,30 wk,M00681269,66,LL,2025-05-01,IR
16,30 wk,M00681270,67,NM,2025-05-01,IR
28,30 wk,M00681271,68,L,2025-05-01,Control
9,30 wk,M00681272,69,R,2025-05-01,Control
17,30 wk,M00681273,70,LL,2025-05-01,IR
32,30 wk,M00681274,71,NM,2025-05-01,IR
1,30 wk,M00681275,72,L,2025-05-02,Control


In [8]:
data = {}

### PSLAX Bmode

keys = [
    "PSLAX_Bmode",
    "PSLAX_Mmode",
    "PSSAX_Mmode",
]

for key in keys:
    # for file in Path("data/InVivo2/blinded_echo").glob(f"202?????_InVivo2_Blinded_*_{key}.csv"):
    for file in Path("data/30-36wk Blinded/blinded_echo").glob(f"202?????_InVivo2_Blinded2_*_{key}.csv"):
        
        date, _, _, image_num, _, _ = file.name.split("_")
        
        image_num = int(image_num)
        
        match key:
            case "PSLAX_Bmode":
                search = '"Measurement","Mode","Parameter","Units",'
                n_lines = 11
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                search = '"Measurement","Mode","Parameter","Units","Avg","STD","Instance 1","Instance 2","Instance 3"'
                n_lines = 6
            case _:
                raise ValueError

        cmds = [
            "grep",
            f"-A{n_lines}",
            search,
            file,
        ]

        result = subprocess.run(cmds, stdout=subprocess.PIPE, text=True)
        df = pd.read_csv(io.StringIO(result.stdout), index_col=False)

        d = data.setdefault(image_num, {})
        
        d.update({
            "Randomised number":int(image_num),
            f"{key}: Date": datetime.strptime(date, "%Y%m%d").date(),
        })

        match key:
            case "PSLAX_Bmode":
                for i,row in df.iterrows():
                    param = row["Parameter"]
                    unit = row["Units"]
                    value = row[-1]
                    d[f"{key}: {param} [{unit}]"] = value
            case "PSLAX_Mmode" | "PSSAX_Mmode":
                for i,row in df.iterrows():
                    param = row["Measurement"]
                    unit = row["Units"]
                    d[f"{key}: {param} Avg [{unit}]"] = row["Avg"]
                    d[f"{key}: {param} STD [{unit}]"] = row["STD"]

#### COMBINE

df2 = pd.DataFrame(data.values())
# df = df.set_index("Randomised number")

#### MAP TO RANDOM NUMBERS

for col in indices:
    df2[col] = df2["Randomised number"].apply(lambda x: mdf2.loc[x,col])

#### SORT

timepoints = ["Baseline", "4 wk", "8 wk", "10 wk", "14 wk", "18 wk", "22 wk", "25 wk", "30 wk", "36 wk"]

df2["Timepoint"] = pd.Categorical(df2["Timepoint"], categories=timepoints, ordered=True)
    
df2 = df2.sort_values(by=indices)
df2 = df2.set_index(indices + ["Randomised number"])

df2

PSLAX_Bmode: Date  \
Timepoint Treatment Rat Animal number Randomised number                     
30 wk     Control   64  M00681267     11                       2025-06-17   
                    65  M00681268     10                       2025-06-17   
                    68  M00681271     28                       2025-06-17   
                    69  M00681272     9                        2025-06-17   
                    72  M00681275     1                        2025-06-17   
                    73  M00681276     22                       2025-06-17   
                    76  M00681279     3                        2025-06-17   
                    77  M00681280     26                       2025-06-17   
          IR        66  M00681269     8                        2025-06-17   
                    67  M00681270     16                       2025-06-17   
                    70  M00681273     17                       2025-06-17   
                    71  M00681274     32                       2025-06-17   
                    74  M00681277     5                        2025-06-17   
                    75  M00681278     14                       2025-06-17   
                    78  M00681281     27                       2025-06-17   
                    79  M00681282     4                        2025-06-17   
36 wk     Control   64  M00681267     15                       2025-06-17   
                    65  M00681268     21                       2025-06-17   
                    68  M00681271     13                       2025-06-17   
                    69  M00681272     18                       2025-06-17   
                    72  M00681275     6                        2025-06-17   
                    73  M00681276     29                       2025-06-17   
                    76  M00681279     24                       2025-06-17   
                    77  M00681280     31                       2025-06-17   
          IR        66  M00681269     12                       2025-06-17   
                    67  M00681270     7                        2025-06-17   
                    70  M00681273     19                       2025-06-17   
                    71  M00681274     23                       2025-06-17   
                    74  M00681277     25                       2025-06-17   
                    75  M00681278     2                        2025-06-17   
                    78  M00681281     30                       2025-06-17   
                    79  M00681282     20                       2025-06-17   

                                                         PSLAX_Bmode: Heart Rate [BPM]  \
Timepoint Treatment Rat Animal number Randomised number                                  
30 wk     Control   64  M00681267     11                                    332.317917   
                    65  M00681268     10                                    336.134454   
                    68  M00681271     28                                    388.538125   
                    69  M00681272     9                                     377.239862   
                    72  M00681275     1                                     325.512003   
                    73  M00681276     22                                    342.563517   
                    76  M00681279     3                                     372.960373   
                    77  M00681280     26                                    366.580113   
          IR        66  M00681269     8                                     391.772772   
                    67  M00681270     16                                    338.504937   
                    70  M00681273     17                                    329.625052   
                    71  M00681274     32                                    342.906129   
                    74  M00681277     5                                     337.695230   
                    75  M00681278     14                                    362.921518 

In [9]:
DF = pd.concat([df1, df2])

In [10]:
DF

PSLAX_Bmode: Date  \
Timepoint Treatment Rat Animal number Randomised number                     
Baseline  Control   64  M00681267     53                       2025-04-28   
                    65  M00681268     47                       2025-04-26   
                    68  M00681271     40                       2025-04-26   
                    69  M00681272     117                      2025-05-02   
                    72  M00681275     76                       2025-04-28   
...                                                                   ...   
36 wk     IR        71  M00681274     23                       2025-06-17   
                    74  M00681277     25                       2025-06-17   
                    75  M00681278     2                        2025-06-17   
                    78  M00681281     30                       2025-06-17   
                    79  M00681282     20                       2025-06-17   

                                                         PSLAX_Bmode: Heart Rate [BPM]  \
Timepoint Treatment Rat Animal number Randomised number                                  
Baseline  Control   64  M00681267     53                                    428.189117   
                    65  M00681268     47                                    435.808970   
                    68  M00681271     40                                    468.566966   
                    69  M00681272     117                                   418.628990   
                    72  M00681275     76                                    399.600400   
...                                                                                ...   
36 wk     IR        71  M00681274     23                                    286.875448   
                    74  M00681277     25                                    299.925019   
                    75  M00681278     2                                     332.042059   
                    78  M00681281     30                                    342.221588   
                    79  M00681282     20                                    353.356890   

                                                         PSLAX_Bmode: Area [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                            
Baseline  Control   64  M00681267     53                               30.421540   
                    65  M00681268     47                               32.815279   
                    68  M00681271     40                               28.106887   
                    69  M00681272     117                              34.847759   
                    72  M00681275     76                               29.239489   
...                                                                          ...   
36 wk     IR        71  M00681274     23                               77.668891   
                    74  M00681277     25                               77.744792   
                    75  M00681278     2                                28.037531   
                    78  M00681281     30                               65.528173   
                    79  M00681282     20                               59.342745   

                                                         PSLAX_Bmode: Area;s [mm2]  \
Timepoint Treatment Rat Animal number Randomised number                              
Baseline  Control   64  M00681267     53                                 26.736552   
                    65  M00681268     47                                 30.244550   
                    68  M00681271     40                                 26.581895   
                    69  M00681272     117                                33.943376   
                    72  M00681275     76                                 27.919859   
...                                                                            ...   
36 wk     IR        71  M00681274     23                                 37.730280   
                

In [11]:
DF.to_excel("InVivo2_echo_processed_36wks.xlsx")